In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader, Dataset
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [3]:
train_data = pd.read_csv("./arxiv_train.csv")
train_data

,Unnamed: 0,abstract,label
0,31716,Automatic meeting analysis is an essential f...,eess
1,89533,We propose a protocol to encode classical bi...,quant-ph
2,82700,A number of physically intuitive results for...,quant-ph
3,78830,In the last decade rare-earth hexaborides ha...,physics
4,94948,We introduce the weak barycenter of a family...,stat
...,...,...,...
79995,27913,"In this paper, the sum secure degrees of fre...",cs
79996,94441,"In areas of application, including actuarial...",stat
79997,33015,Failure detection is employed in the industr...,eess
79998,942,As part of the ongoing effort to characteriz...,astro-ph


In [4]:
# Load the train set
test_data = pd.read_csv("./arxiv_test.csv")
test_data

,Unnamed: 0,abstract,label
0,64481,We describe a shape derivative approach to p...,math
1,48104,We study displaced signatures of sneutrino p...,hep-ph
2,48233,High precision studies of Beyond-Standard-Mo...,hep-ph
3,49026,We find that a class of models of MeV-GeV da...,hep-ph
4,37957,Knowledge of power grid's topology during ca...,eess
...,...,...,...
19995,50391,We explore the dynamics of a simple class of...,hep-th
19996,63534,In this paper one construction of compositio...,math
19997,16712,The Random-First-Order-Transition theory of ...,cond-mat
19998,6596,Accurate chemical abundance measurements of ...,astro-ph


In [5]:
count_vectorizer = CountVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [6]:
X_train_count = count_vectorizer.fit_transform(train_data["abstract"]).toarray()
X_test_count = count_vectorizer.transform(test_data["abstract"]).toarray()

In [7]:
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data["abstract"]).toarray()
X_test_tfidf = tfidf_vectorizer.transform(test_data["abstract"]).toarray()

In [8]:
label_encoder = {label: idx for idx, label in enumerate(train_data["label"].unique())}
train_data["label_encoded"] = train_data["label"].map(label_encoder)
test_data["label_encoded"] = test_data["label"].map(label_encoder)

In [9]:
y_train = torch.tensor(train_data["label_encoded"].values)
y_test = torch.tensor(test_data["label_encoded"].values)

In [10]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [11]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, activation=nn.ReLU()):
        super(FFNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_layers = nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.hidden_layers.append(nn.Linear(input_size, hidden_size))
            elif i == num_layers - 1:
                self.hidden_layers.append(nn.Linear(hidden_size, output_size))
            else:
                self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
            self.hidden_layers.append(activation)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        return x



In [12]:
# Function to train the model
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss}")

In [13]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.float())
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

In [14]:
# Define train and test datasets
train_dataset_count = CustomDataset(X_train_count, y_train)
test_dataset_count = CustomDataset(X_test_count, y_test)

train_dataset_tfidf = CustomDataset(X_train_tfidf, y_train)
test_dataset_tfidf = CustomDataset(X_test_tfidf, y_test)

# Define data loaders
train_loader_count = DataLoader(train_dataset_count, batch_size=64, shuffle=True)
test_loader_count = DataLoader(test_dataset_count, batch_size=64, shuffle=False)

train_loader_tfidf = DataLoader(train_dataset_tfidf, batch_size=64, shuffle=True)
test_loader_tfidf = DataLoader(test_dataset_tfidf, batch_size=64, shuffle=False)

# Evaluating different number of layers 

In [14]:
# Train and evaluate FFNN with CountVectorizer and different numbers of layers
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
for num_layers in [1, 2, 3]:
    print(f"Training FFNN with CountVectorizer and {num_layers} layers")
    ffnn_count = FFNN(input_size_count, 256, output_size_count, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
    train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_count, test_loader_count)
    print()

Training FFNN with CountVectorizer and 1 layers
Epoch [1/10], Loss: 743.0623899400234
Epoch [2/10], Loss: 445.8628660440445
Epoch [3/10], Loss: 263.4596190750599
Epoch [4/10], Loss: 114.98958802036941
Epoch [5/10], Loss: 38.809479392599314
Epoch [6/10], Loss: 14.333205292350613
Epoch [7/10], Loss: 7.945223589427769
Epoch [8/10], Loss: 20.164672709885053
Epoch [9/10], Loss: 16.07088966111769
Epoch [10/10], Loss: 7.142239848835743
Evaluation on Test Set:
Accuracy: 0.8107, Precision: 0.8109409515555879, Recall: 0.8104510494535951, F1-Score: 0.8096136114078154

Training FFNN with CountVectorizer and 2 layers
Epoch [1/10], Loss: 749.5075382143259
Epoch [2/10], Loss: 440.41998664289713
Epoch [3/10], Loss: 258.40501895174384
Epoch [4/10], Loss: 110.69346080627292
Epoch [5/10], Loss: 52.09413174400106
Epoch [6/10], Loss: 48.67403827956878
Epoch [7/10], Loss: 33.54433995686122
Epoch [8/10], Loss: 30.207818542796304
Epoch [9/10], Loss: 32.536002864377224
Epoch [10/10], Loss: 25.806746830567135
E

In [20]:
input_size_tfidf = X_train_tfidf.shape[1]
output_size_tfidf = len(label_encoder)
for num_layers in [1, 2, 3]:
    print(f"Training FFNN with TF-IDF and {num_layers} layers")
    ffnn_tfidf = FFNN(input_size_tfidf, 256, output_size_tfidf, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_tfidf.parameters(), lr=0.001)
    train_model(ffnn_tfidf, criterion, optimizer, train_loader_tfidf, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_tfidf, test_loader_tfidf)
    print()

Training FFNN with TF-IDF and 1 layers
Epoch [1/10], Loss: 5326.840245485306
Epoch [2/10], Loss: 3824.8796890974045
Epoch [3/10], Loss: 3184.4817782640457
Epoch [4/10], Loss: 3002.0328738689423
Epoch [5/10], Loss: 2892.8296970129013
Epoch [6/10], Loss: 2819.685179710388
Epoch [7/10], Loss: 2767.6214294433594
Epoch [8/10], Loss: 2728.7652148008347
Epoch [9/10], Loss: 2698.5428136587143
Epoch [10/10], Loss: 2674.2725652456284
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.60995, Precision: 0.4508947859006704, Recall: 0.6116196951717745, F1-Score: 0.5135931465719163

Training FFNN with TF-IDF and 2 layers
Epoch [1/10], Loss: 1515.5565091967583
Epoch [2/10], Loss: 1282.4970963001251
Epoch [3/10], Loss: 1224.7731808423996
Epoch [4/10], Loss: 1178.9496647119522
Epoch [5/10], Loss: 1134.6555883586407
Epoch [6/10], Loss: 1090.07475528121
Epoch [7/10], Loss: 1048.1092193722725
Epoch [8/10], Loss: 1011.3235712349415
Epoch [9/10], Loss: 981.8349013328552
Epoch [10/10], Loss: 960.0872549414635
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6571, Precision: 0.5788497089397758, Recall: 0.6565813106362854, F1-Score: 0.6075712012055713

Training FFNN with TF-IDF and 3 layers
Epoch [1/10], Loss: 1682.6027935743332
Epoch [2/10], Loss: 1506.8588567376137
Epoch [3/10], Loss: 1428.2502170205116
Epoch [4/10], Loss: 1336.810312628746
Epoch [5/10], Loss: 1258.6320641040802
Epoch [6/10], Loss: 1223.6720420718193
Epoch [7/10], Loss: 1216.6805083155632
Epoch [8/10], Loss: 1218.678000152111
Epoch [9/10], Loss: 1214.6673249602318
Epoch [10/10], Loss: 1213.8080894947052
Evaluation on Test Set:
Accuracy: 0.57375, Precision: 0.5084068580230362, Recall: 0.5751658508275816, F1-Score: 0.522698408944996



C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Evaluating Activation Function for Count Vectorizer feature detection method

In [18]:
# Train and evaluate FFNN with CountVectorizer and different activation function
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
activation_functions = [nn.LeakyReLU(), nn.ELU(), nn.Sigmoid(), nn.Softmax(dim=1)]

for activation_function in activation_functions:
    print(f"Training FFNN with CountVectorizer and {activation_function.__class__.__name__} as an activation function")
    ffnn_count = FFNN(input_size_count, 256, output_size_count, num_layers=2, activation=activation_function)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
    train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_count, test_loader_count)
    print()


Training FFNN with CountVectorizer and LeakyReLU as an activation function
Epoch [1/10], Loss: 779.1226999759674
Epoch [2/10], Loss: 461.2558950930834
Epoch [3/10], Loss: 290.4565050601959
Epoch [4/10], Loss: 142.1204579025507
Epoch [5/10], Loss: 54.91493967361748
Epoch [6/10], Loss: 22.617049310123548
Epoch [7/10], Loss: 17.038059719197918
Epoch [8/10], Loss: 20.81000182213029
Epoch [9/10], Loss: 12.523910178308142
Epoch [10/10], Loss: 10.43505616929906
Evaluation on Test Set:
Accuracy: 0.8091, Precision: 0.8084935102365746, Recall: 0.8085558081893665, F1-Score: 0.8082112300010884

Training FFNN with CountVectorizer and ELU as an activation function
Epoch [1/10], Loss: 733.471880197525
Epoch [2/10], Loss: 483.33389146625996
Epoch [3/10], Loss: 373.9589847549796
Epoch [4/10], Loss: 272.5039286017418
Epoch [5/10], Loss: 177.92759406752884
Epoch [6/10], Loss: 97.2131339032203
Epoch [7/10], Loss: 48.75164867145941
Epoch [8/10], Loss: 25.524274023831822
Epoch [9/10], Loss: 18.6469395519234

In [48]:
activation_functions = [nn.LeakyReLU(), nn.ELU(), nn.Sigmoid(), nn.Softmax(dim=1)]

for activation_function in activation_functions:
    print(f"Training FFNN with tfidf and {activation_function.__class__.__name__} as an activation function")
    ffnn_tfidf = FFNN(input_size_tfidf, 256, output_size_tfidf, num_layers=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_tfidf.parameters(), lr=0.001)
    train_model(ffnn_tfidf, criterion, optimizer, train_loader_tfidf, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_tfidf, test_loader_tfidf)
    print()

Training FFNN with tfidf and LeakyReLU as an activation function
Epoch [1/10], Loss: 1749.8307220339775
Epoch [2/10], Loss: 1550.7487240433693
Epoch [3/10], Loss: 1499.347153186798
Epoch [4/10], Loss: 1456.0741328001022
Epoch [5/10], Loss: 1411.1687846183777
Epoch [6/10], Loss: 1365.7961130142212
Epoch [7/10], Loss: 1321.3161405324936
Epoch [8/10], Loss: 1283.5120163559914
Epoch [9/10], Loss: 1255.94439214468
Epoch [10/10], Loss: 1236.8211332559586
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.55025, Precision: 0.4722979387091396, Recall: 0.5528656732419206, F1-Score: 0.4977211554277593

Training FFNN with tfidf and ELU as an activation function
Epoch [1/10], Loss: 1503.7976242303848
Epoch [2/10], Loss: 1279.82207518816
Epoch [3/10], Loss: 1221.6461164355278
Epoch [4/10], Loss: 1175.0153464078903
Epoch [5/10], Loss: 1126.505851328373
Epoch [6/10], Loss: 1078.1066450178623
Epoch [7/10], Loss: 1031.5820057988167
Epoch [8/10], Loss: 989.6726938486099
Epoch [9/10], Loss: 957.9369266033173
Epoch [10/10], Loss: 935.7010561823845
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.59085, Precision: 0.5018227476802009, Recall: 0.5929940277789638, F1-Score: 0.5275902386624757

Training FFNN with tfidf and Sigmoid as an activation function
Epoch [1/10], Loss: 2028.3624507188797
Epoch [2/10], Loss: 1909.6063405275345
Epoch [3/10], Loss: 1878.682795882225
Epoch [4/10], Loss: 1853.0613781809807
Epoch [5/10], Loss: 1830.747833609581
Epoch [6/10], Loss: 1810.5023810863495
Epoch [7/10], Loss: 1794.5962591171265
Epoch [8/10], Loss: 1784.6853432655334
Epoch [9/10], Loss: 1777.7796366810799
Epoch [10/10], Loss: 1774.8702552318573
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4536, Precision: 0.3546719341470822, Recall: 0.4535931848807375, F1-Score: 0.37539067898747575

Training FFNN with tfidf and Softmax as an activation function
Epoch [1/10], Loss: 1684.201228916645
Epoch [2/10], Loss: 1496.6884635686874
Epoch [3/10], Loss: 1447.003049492836
Epoch [4/10], Loss: 1404.558314859867
Epoch [5/10], Loss: 1361.492532670498
Epoch [6/10], Loss: 1318.2550931572914
Epoch [7/10], Loss: 1278.500360250473
Epoch [8/10], Loss: 1246.0123883485794
Epoch [9/10], Loss: 1222.7079489827156
Epoch [10/10], Loss: 1207.5390406847
Evaluation on Test Set:
Accuracy: 0.52955, Precision: 0.4262029780118576, Recall: 0.5315308066577746, F1-Score: 0.4542369088206838



C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
input_size_tfidf = X_train_tfidf.shape[1]
output_size_tfidf = len(label_encoder)
activation_functions = [nn.LeakyReLU(), nn.ELU(), nn.Sigmoid(), nn.Softmax(dim=1)]

for activation_function in activation_functions:
    print(f"Training FFNN with tfidf and {activation_function.__class__.__name__} as an activation function")
    ffnn_tfidf = FFNN(input_size_tfidf, 256, output_size_tfidf, num_layers=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ffnn_tfidf.parameters(), lr=0.001)
    train_model(ffnn_tfidf, criterion, optimizer, train_loader_tfidf, num_epochs=10)
    print("Evaluation on Test Set:")
    evaluate_model(ffnn_tfidf, test_loader_tfidf)
    print()

Training FFNN with tfidf and LeakyReLU as an activation function
Epoch [1/10], Loss: 1989.0983240008354
Epoch [2/10], Loss: 1353.0579169988632
Epoch [3/10], Loss: 1045.6689096987247
Epoch [4/10], Loss: 918.1834497153759
Epoch [5/10], Loss: 860.5946475565434
Epoch [6/10], Loss: 805.3669632673264
Epoch [7/10], Loss: 753.17565099895
Epoch [8/10], Loss: 705.7292978316545
Epoch [9/10], Loss: 669.6869882196188
Epoch [10/10], Loss: 643.85013666749
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.67525, Precision: 0.5958855356498138, Recall: 0.6769820138908375, F1-Score: 0.622319832409534

Training FFNN with tfidf and ELU as an activation function
Epoch [1/10], Loss: 1741.1844599843025
Epoch [2/10], Loss: 1542.819808781147
Epoch [3/10], Loss: 1493.7148686647415
Epoch [4/10], Loss: 1455.0220013856888
Epoch [5/10], Loss: 1417.0658017992973
Epoch [6/10], Loss: 1376.7458293437958
Epoch [7/10], Loss: 1336.772608935833
Epoch [8/10], Loss: 1299.9106670618057
Epoch [9/10], Loss: 1269.2381512522697
Epoch [10/10], Loss: 1246.8699571490288
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5538, Precision: 0.4752740156301395, Recall: 0.5567484209814866, F1-Score: 0.5018888406457622

Training FFNN with tfidf and Sigmoid as an activation function
Epoch [1/10], Loss: 1697.6441856622696
Epoch [2/10], Loss: 1502.6167083978653
Epoch [3/10], Loss: 1452.2089486718178
Epoch [4/10], Loss: 1411.4019531607628
Epoch [5/10], Loss: 1372.9568030238152
Epoch [6/10], Loss: 1334.5735786557198
Epoch [7/10], Loss: 1298.6759565472603
Epoch [8/10], Loss: 1267.5561074614525
Epoch [9/10], Loss: 1245.2560103535652
Epoch [10/10], Loss: 1229.4111191034317
Evaluation on Test Set:


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.58055, Precision: 0.48890891388077584, Recall: 0.5816266085872539, F1-Score: 0.5176493215945227

Training FFNN with tfidf and Softmax as an activation function
Epoch [1/10], Loss: 1077.9348317086697
Epoch [2/10], Loss: 765.788494348526
Epoch [3/10], Loss: 690.202105641365
Epoch [4/10], Loss: 629.3187815696001
Epoch [5/10], Loss: 568.375945404172
Epoch [6/10], Loss: 508.4011527746916
Epoch [7/10], Loss: 450.0943462923169
Epoch [8/10], Loss: 400.73719269037247
Epoch [9/10], Loss: 363.0493036210537
Epoch [10/10], Loss: 337.2618256136775
Evaluation on Test Set:
Accuracy: 0.7244, Precision: 0.6799357441171696, Recall: 0.724543853929455, F1-Score: 0.696241392087842



C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Model for evaluating batch size

In [16]:
num_epochs = 10
learning_rate = 0.001
batch_sizes = [32, 64, 128, 256] 
results = {}
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
for batch_size in batch_sizes:
    print(f"Evaluating with batch size: {batch_size}")
    
    model = FFNN(input_size_count, 256, output_size_count, num_layers=2, activation=nn.ReLU())
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for batch_size in batch_sizes: 
        train_loader_count = DataLoader(train_dataset_count, batch_size, shuffle=True)
        test_loader_count = DataLoader(test_dataset_count, batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader_count:
            optimizer.zero_grad()
            inputs = inputs.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        accuracy = evaluate_model(model, test_loader_count)
    results[batch_size] = accuracy
    print(f"Accuracy with batch size {batch_size}: {accuracy}")



Evaluating with batch size: 32


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6639, Precision: 0.5820561220094096, Recall: 0.661727117961166, F1-Score: 0.610218188955114


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.65795, Precision: 0.585144779967459, Recall: 0.6556482963549323, F1-Score: 0.6068712857978127


...

Accuracy: 0.7233, Precision: 0.6779000698842585, Recall: 0.7237604277731892, F1-Score: 0.6949232042368994


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7245, Precision: 0.6783932827333877, Recall: 0.724949851934195, F1-Score: 0.6957533035886423


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7251, Precision: 0.6807897722349743, Recall: 0.7256239640678585, F1-Score: 0.6967675600669601
Accuracy with batch size 256: None
Evaluating with batch size: 128
Accuracy: 0.82235, Precision: 0.8254322844562866, Recall: 0.8219659914828418, F1-Score: 0.822201449845289
Accuracy: 0.8198, Precision: 0.8230098011811297, Recall: 0.8194194231566418, F1-Score: 0.8204722047323697
Accuracy: 0.8194, Precision: 0.8200415861795639, Recall: 0.8190351461096149, F1-Score: 0.8191175423765328
Accuracy: 0.8158, Precision: 0.8149317206989928, Recall: 0.8153869276845478, F1-Score: 0.8146790723810209
Accuracy: 0.81365, Precision: 0.8125185260552639, Recall: 0.8131683659793925, F1-Score: 0.8126635481305085
Accuracy: 0.8114, Precision: 0.8113390763656299, Recall: 0.811014480530079, F1-Score: 0.8109956760827475
Accuracy: 0.8088, Precision: 0.8082202132143383, Recall: 0.8083993215311743, F1-Score: 0.8080166066546312
Accuracy: 0.80815, Precision: 0.8084284901342649, Recall: 0.8077816209117982, F1-Scor

In [17]:
num_epochs = 10
learning_rate = 0.001
batch_sizes = [32, 64, 128, 256] 
results = {}
input_size_count = X_train_tfidf.shape[1]
output_size_count = len(label_encoder)
for batch_size in batch_sizes:
    print(f"Evaluating with batch size: {batch_size}")
    
    model = FFNN(input_size_count, 256, output_size_count, num_layers=2, activation=nn.ReLU())
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for batch_size in batch_sizes: 
        train_loader_count = DataLoader(train_dataset_count, batch_size, shuffle=True)
        test_loader_count = DataLoader(test_dataset_count, batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader_count:
            optimizer.zero_grad()
            inputs = inputs.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        accuracy = evaluate_model(model, test_loader_count)
    results[batch_size] = accuracy
    print(f"Accuracy with batch size {batch_size}: {accuracy}")


Evaluating with batch size: 32


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7785, Precision: 0.7068571195633515, Recall: 0.7771184342932531, F1-Score: 0.7391880127879042


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7776, Precision: 0.7146890539093732, Recall: 0.7763764289729352, F1-Score: 0.7400166903227179


...

C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.68245, Precision: 0.5983047705647172, Recall: 0.6844387395215084, F1-Score: 0.6288824947881915


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6824, Precision: 0.5995928764105468, Recall: 0.6844208548521176, F1-Score: 0.6293758897544344
Accuracy: 0.68275, Precision: 0.5984525250922998, Recall: 0.6847429480259987, F1-Score: 0.6291601255434467
Accuracy with batch size 256: None


C:\Users\SaharKadkhodaMasoumA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Model for regularization

In [18]:


class FFNN_regularization(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(FFNN_regularization, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [30]:
input_size_count = X_train_count.shape[1]
output_size_count = len(label_encoder)
ffnn_count = FFNN_regularization(input_size_count, 256, output_size_count)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
print("Evaluation on Test Set:")
evaluate_model(ffnn_count, test_loader_count)
print()

Epoch [1/10], Loss: 803.1190403103828
Epoch [2/10], Loss: 528.7452176064253
Epoch [3/10], Loss: 412.16740706562996
Epoch [4/10], Loss: 325.4513051956892
Epoch [5/10], Loss: 253.0888447985053
Epoch [6/10], Loss: 201.6329963169992
Epoch [7/10], Loss: 162.31344585865736
Epoch [8/10], Loss: 136.57539752218872
Epoch [9/10], Loss: 113.82700414489955
Epoch [10/10], Loss: 101.64528997661546
Evaluation on Test Set:
Accuracy: 0.8227, Precision: 0.823572286469816, Recall: 0.8224513551533272, F1-Score: 0.8224330314293568



In [19]:
input_size_count = X_train_tfidf.shape[1]
output_size_count = len(label_encoder)
ffnn_count = FFNN_regularization(input_size_count, 256, output_size_count)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ffnn_count.parameters(), lr=0.001)
train_model(ffnn_count, criterion, optimizer, train_loader_count, num_epochs=10)
print("Evaluation on Test Set:")
evaluate_model(ffnn_count, test_loader_count)
print()

Epoch [1/10], Loss: 222.48153260350227
Epoch [2/10], Loss: 140.18351757526398
Epoch [3/10], Loss: 114.07512375712395
Epoch [4/10], Loss: 93.81218022108078
Epoch [5/10], Loss: 76.5084165930748
Epoch [6/10], Loss: 61.93401847034693
Epoch [7/10], Loss: 49.72272405028343
Epoch [8/10], Loss: 40.39252459257841
Epoch [9/10], Loss: 33.31942366436124
Epoch [10/10], Loss: 27.530828561633825
Evaluation on Test Set:
Accuracy: 0.82535, Precision: 0.8244408031734916, Recall: 0.8249560068774324, F1-Score: 0.8246063205706873

